<a href="https://colab.research.google.com/github/BelloBer/Landslide-Detection/blob/main/complete2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OPTIMIZED CODE
EXPECTED OUTPUT
* higher f1 score for landslide (above 0.68)
* Balanced F1 scores > 0.9 for both classes
* Faster convergence due to better architecture
* More stable training with residual connections
* Better generalization from improved regularization

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, classification_report, confusion_matrix, precision_recall_curve
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Dense, Flatten, Conv2D, MaxPooling2D, Dropout,
                                   BatchNormalization, Input, GlobalAveragePooling2D,
                                   SeparableConv2D, Add, Activation)
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import datetime
from collections import Counter
import gc
from scipy import ndimage

# Updated Configuration for 64x64 images
CONFIG = {
    'TRAIN_DATA': '/content/drive/My Drive/train_data/',
    'TEST_DATA': '/content/drive/My Drive/test_data/',
    'TRAIN_CSV': "/content/drive/My Drive/train_data/Train.csv",
    'TEST_CSV': "/content/drive/My Drive/test_data/Test.csv",
    'BATCH_SIZE': 16,
    'IMG_SIZE': 64,  # Changed from 256 to 64
    'CHANNELS': 12,
    'EPOCHS': 100,
    'LEARNING_RATE': 0.0001,
    'PATIENCE': 15,
    'VALIDATION_SPLIT': 0.2
}

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class DataProcessor:
    """Handles data loading and preprocessing with improved normalization"""

    @staticmethod
    def load_and_normalize_image(image_id, folder_path):
        """Enhanced image loading with better SAR processing"""
        try:
            image_path = os.path.join(folder_path, f"{image_id}.npy")
            if not os.path.exists(image_path):
                return None

            img = np.load(image_path)

            if len(img.shape) != 3 or img.shape[2] != 12:
                return None

            # Check actual image size and log it for debugging
            actual_size = img.shape[:2]
            if actual_size != (CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']):
                print(f"Warning: Image {image_id} has size {actual_size}, expected {(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'])}")
                # Resize if needed
                from tensorflow.keras.preprocessing.image import smart_resize
                img_resized = np.zeros((CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], CONFIG['CHANNELS']))
                for ch in range(CONFIG['CHANNELS']):
                    img_resized[:, :, ch] = smart_resize(img[:, :, ch:ch+1], (CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'])).squeeze()
                img = img_resized

            img_normalized = np.zeros_like(img, dtype=np.float32)

            # Optical bands (0-3) - Enhanced normalization
            for band in range(4):
                band_data = img[:, :, band].astype(np.float32)

                # Remove outliers using percentile clipping
                p2, p98 = np.percentile(band_data, [2, 98])
                band_data = np.clip(band_data, p2, p98)

                # Robust normalization
                if p98 > p2:
                    img_normalized[:, :, band] = (band_data - p2) / (p98 - p2)
                else:
                    img_normalized[:, :, band] = 0.5

            # SAR bands (4-11) - Improved SAR processing
            for band in range(4, 12):
                sar_data = img[:, :, band].astype(np.float32)

                # Convert to dB with proper handling
                sar_positive = np.abs(sar_data)
                sar_positive = np.maximum(sar_positive, 1e-12)
                sar_db = 10 * np.log10(sar_positive + 1e-12)

                # Clip extreme values
                sar_db = np.clip(sar_db, -50, 10)

                # Normalize to [0, 1]
                img_normalized[:, :, band] = (sar_db + 50) / 60

            return img_normalized

        except Exception as e:
            print(f"Error loading {image_id}: {str(e)}")
            return None

class MultiChannelAugmenter:
    """Custom augmentation for multi-channel images (12 channels) optimized for 64x64"""

    def __init__(self, rotation_range=30, width_shift_range=0.2, height_shift_range=0.2,
                 zoom_range=0.2, horizontal_flip=True, vertical_flip=True,
                 brightness_range=[0.9, 1.1], noise_factor=0.03):
        # Reduced augmentation ranges for smaller 64x64 images
        self.rotation_range = rotation_range
        self.width_shift_range = width_shift_range
        self.height_shift_range = height_shift_range
        self.zoom_range = zoom_range
        self.horizontal_flip = horizontal_flip
        self.vertical_flip = vertical_flip
        self.brightness_range = brightness_range
        self.noise_factor = noise_factor

    def random_rotation(self, image, angle_range):
        """Apply random rotation to all channels"""
        angle = np.random.uniform(-angle_range, angle_range)
        rotated = np.zeros_like(image)
        for i in range(image.shape[2]):
            rotated[:, :, i] = ndimage.rotate(image[:, :, i], angle, reshape=False, mode='reflect')
        return rotated

    def random_shift(self, image, width_range, height_range):
        """Apply random translation to all channels"""
        h, w = image.shape[:2]
        dx = int(np.random.uniform(-width_range, width_range) * w)
        dy = int(np.random.uniform(-height_range, height_range) * h)

        shifted = np.zeros_like(image)
        for i in range(image.shape[2]):
            shifted[:, :, i] = ndimage.shift(image[:, :, i], [dy, dx], mode='reflect')
        return shifted

    def random_zoom(self, image, zoom_range):
        """Apply random zoom to all channels"""
        zoom_factor = np.random.uniform(1-zoom_range, 1+zoom_range)
        zoomed = np.zeros_like(image)

        for i in range(image.shape[2]):
            zoomed_channel = ndimage.zoom(image[:, :, i], zoom_factor, mode='reflect')

            # Crop or pad to maintain original size
            orig_h, orig_w = image.shape[:2]
            if zoom_factor > 1:  # Crop
                h, w = zoomed_channel.shape
                start_h = max(0, (h - orig_h) // 2)
                start_w = max(0, (w - orig_w) // 2)
                end_h = min(h, start_h + orig_h)
                end_w = min(w, start_w + orig_w)
                cropped = zoomed_channel[start_h:end_h, start_w:end_w]

                # Pad if necessary
                if cropped.shape != (orig_h, orig_w):
                    padded = np.zeros((orig_h, orig_w))
                    h_pad = (orig_h - cropped.shape[0]) // 2
                    w_pad = (orig_w - cropped.shape[1]) // 2
                    padded[h_pad:h_pad+cropped.shape[0], w_pad:w_pad+cropped.shape[1]] = cropped
                    zoomed[:, :, i] = padded
                else:
                    zoomed[:, :, i] = cropped

            elif zoom_factor < 1:  # Pad
                h, w = zoomed_channel.shape
                padded = np.zeros((orig_h, orig_w))
                pad_h = (orig_h - h) // 2
                pad_w = (orig_w - w) // 2
                padded[pad_h:pad_h+h, pad_w:pad_w+w] = zoomed_channel
                zoomed[:, :, i] = padded
            else:
                zoomed[:, :, i] = zoomed_channel

        return zoomed

    def random_flip(self, image):
        """Apply random flips to all channels"""
        flipped = image.copy()

        if self.horizontal_flip and np.random.random() > 0.5:
            flipped = np.fliplr(flipped)

        if self.vertical_flip and np.random.random() > 0.5:
            flipped = np.flipud(flipped)

        return flipped

    def random_brightness(self, image):
        """Apply brightness adjustment only to optical bands (0-3)"""
        augmented = image.copy()

        # Only apply to optical bands (channels 0-3)
        brightness_factor = np.random.uniform(self.brightness_range[0], self.brightness_range[1])
        for i in range(4):  # Only optical bands
            augmented[:, :, i] = np.clip(augmented[:, :, i] * brightness_factor, 0, 1)

        return augmented

    def add_noise(self, image):
        """Add small amount of noise"""
        noise = np.random.normal(0, self.noise_factor, image.shape)
        return np.clip(image + noise, 0, 1)

    def random_transform(self, image):
        """Apply random combination of augmentations"""
        augmented = image.copy()

        # Apply transformations with certain probabilities
        if np.random.random() > 0.5:
            augmented = self.random_rotation(augmented, self.rotation_range)

        if np.random.random() > 0.5:
            augmented = self.random_shift(augmented, self.width_shift_range, self.height_shift_range)

        if np.random.random() > 0.3:
            augmented = self.random_zoom(augmented, self.zoom_range)

        if np.random.random() > 0.5:
            augmented = self.random_flip(augmented)

        if np.random.random() > 0.3:
            augmented = self.random_brightness(augmented)

        if np.random.random() > 0.7:
            augmented = self.add_noise(augmented)

        return augmented

class BalancedDataGenerator(Sequence):
    """Improved data generator with custom multi-channel augmentation"""

    def __init__(self, image_ids, labels, folder_path, batch_size=16,
                 augment=False, shuffle=True, balance_classes=True):
        super().__init__()

        self.image_ids = np.array(image_ids)
        self.labels = np.array(labels)
        self.folder_path = folder_path
        self.batch_size = batch_size
        self.augment = augment
        self.shuffle = shuffle
        self.balance_classes = balance_classes

        # Validate files
        self.valid_indices = self._validate_files()
        if len(self.valid_indices) == 0:
            raise FileNotFoundError(f"No valid files found in {folder_path}")

        # Create class-balanced indices if needed
        if self.balance_classes:
            self._create_balanced_indices()
        else:
            self.balanced_indices = self.valid_indices.copy()

        self.on_epoch_end()

        # Use custom multi-channel augmenter
        if self.augment:
            self.augmenter = MultiChannelAugmenter(
                rotation_range=30,
                width_shift_range=0.2,
                height_shift_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                vertical_flip=True,
                brightness_range=[0.9, 1.1],
                noise_factor=0.02
            )

    def _validate_files(self):
        """Validate file existence and readability"""
        valid_indices = []
        for idx, img_id in enumerate(self.image_ids):
            file_path = os.path.join(self.folder_path, f"{img_id}.npy")
            if os.path.exists(file_path):
                try:
                    test_img = np.load(file_path)
                    if len(test_img.shape) == 3 and test_img.shape[2] == 12:
                        valid_indices.append(idx)
                except:
                    continue
        return np.array(valid_indices)

    def _create_balanced_indices(self):
        """Create balanced sampling indices"""
        # Separate indices by class
        valid_labels = self.labels[self.valid_indices]
        class_0_indices = self.valid_indices[valid_labels == 0]
        class_1_indices = self.valid_indices[valid_labels == 1]

        # Oversample minority class
        min_class_size = min(len(class_0_indices), len(class_1_indices))
        max_class_size = max(len(class_0_indices), len(class_1_indices))

        # Create balanced dataset by oversampling
        if len(class_0_indices) < len(class_1_indices):
            # Oversample class 0
            oversample_indices = np.random.choice(class_0_indices,
                                                len(class_1_indices) - len(class_0_indices))
            class_0_indices = np.concatenate([class_0_indices, oversample_indices])
        else:
            # Oversample class 1
            oversample_indices = np.random.choice(class_1_indices,
                                                len(class_0_indices) - len(class_1_indices))
            class_1_indices = np.concatenate([class_1_indices, oversample_indices])

        self.balanced_indices = np.concatenate([class_0_indices, class_1_indices])
        print(f"Balanced dataset: {len(self.balanced_indices)} samples")

    def __len__(self):
        return int(np.ceil(len(self.balanced_indices) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.balanced_indices[idx*self.batch_size:(idx+1)*self.batch_size]

        batch_images = []
        batch_labels = []

        for i in batch_indices:
            img_id = self.image_ids[i]
            label = self.labels[i]

            img = DataProcessor.load_and_normalize_image(img_id, self.folder_path)
            if img is not None:
                if self.augment:
                    img = self.augmenter.random_transform(img)
                batch_images.append(img)
                batch_labels.append(label)

        # Ensure we have a full batch
        while len(batch_images) < len(batch_indices) and len(batch_images) > 0:
            rand_idx = np.random.randint(0, len(batch_images))
            batch_images.append(batch_images[rand_idx])
            batch_labels.append(batch_labels[rand_idx])

        return np.array(batch_images), np.array(batch_labels, dtype=np.float32)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.balanced_indices)

class ModelBuilder:
    """Builds improved CNN architecture optimized for 64x64 images"""

    @staticmethod
    def residual_block(x, filters, kernel_size=3):
        """Residual block for better gradient flow"""
        shortcut = x

        x = SeparableConv2D(filters, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = SeparableConv2D(filters, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)

        # Adjust shortcut if needed
        if shortcut.shape[-1] != filters:
            shortcut = Conv2D(filters, 1, padding='same')(shortcut)
            shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = Activation('relu')(x)
        return x

    @staticmethod
    def build_improved_model(input_shape):
        """Build improved CNN optimized for 64x64 input"""
        inputs = Input(shape=input_shape)

        # Initial convolution - smaller stride for 64x64 input
        x = Conv2D(32, (5, 5), strides=1, padding='same')(inputs)  # Changed stride from 2 to 1
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2, 2))(x)  # 64x64 -> 32x32

        # Residual blocks with appropriate pooling for smaller input
        x = ModelBuilder.residual_block(x, 64)
        x = MaxPooling2D((2, 2))(x)  # 32x32 -> 16x16
        x = Dropout(0.25)(x)

        x = ModelBuilder.residual_block(x, 128)
        x = MaxPooling2D((2, 2))(x)  # 16x16 -> 8x8
        x = Dropout(0.25)(x)

        x = ModelBuilder.residual_block(x, 256)
        x = MaxPooling2D((2, 2))(x)  # 8x8 -> 4x4
        x = Dropout(0.3)(x)

        # Additional residual block since we have fewer pooling layers
        x = ModelBuilder.residual_block(x, 512)
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)

        # Classification head - adjusted for smaller feature maps
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)

        x = Dense(128, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)

        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs, outputs)
        return model

# Custom metrics
def precision_m(y_true, y_pred):
    y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
    true_positives = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
    predicted_positives = tf.reduce_sum(y_pred)
    return true_positives / (predicted_positives + tf.keras.backend.epsilon())

def recall_m(y_true, y_pred):
    y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
    true_positives = tf.reduce_sum(tf.cast(y_true * y_pred, tf.float32))
    possible_positives = tf.reduce_sum(y_true)
    return true_positives / (possible_positives + tf.keras.backend.epsilon())

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

# Improved focal loss
def focal_loss(alpha=0.7, gamma=2.0):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())

        # Calculate focal loss
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        focal_weight = y_true * (1 - y_pred) ** gamma + (1 - y_true) * y_pred ** gamma

        # Binary crossentropy
        bce = -(y_true * tf.math.log(y_pred) + (1 - y_true) * tf.math.log(1 - y_pred))

        return tf.reduce_mean(alpha_factor * focal_weight * bce)

    return focal_loss_fixed

# Main execution
def main():
    print("Starting optimized landslide detection training...")

    # Load data
    train_df = pd.read_csv(CONFIG['TRAIN_CSV'])
    print(f"Loaded {len(train_df)} training samples")
    print(f"Class distribution: {train_df['label'].value_counts().to_dict()}")

    # Check actual image sizes
    print("Checking image sizes...")
    sample_ids = train_df['ID'].values[:5]  # Check first 5 images
    for img_id in sample_ids:
        img_path = os.path.join(CONFIG['TRAIN_DATA'], f"{img_id}.npy")
        if os.path.exists(img_path):
            img = np.load(img_path)
            print(f"Image {img_id}: shape {img.shape}")
        else:
            print(f"Image {img_id}: not found")

    # Stratified split
    train_idx, val_idx = train_test_split(
        np.arange(len(train_df)),
        test_size=CONFIG['VALIDATION_SPLIT'],
        random_state=42,
        stratify=train_df['label']
    )

    # Create generators
    train_gen = BalancedDataGenerator(
        image_ids=train_df['ID'].values[train_idx],
        labels=train_df['label'].values[train_idx],
        folder_path=CONFIG['TRAIN_DATA'],
        batch_size=CONFIG['BATCH_SIZE'],
        augment=True,
        balance_classes=True
    )

    val_gen = BalancedDataGenerator(
        image_ids=train_df['ID'].values[val_idx],
        labels=train_df['label'].values[val_idx],
        folder_path=CONFIG['TRAIN_DATA'],
        batch_size=CONFIG['BATCH_SIZE'],
        augment=False,
        balance_classes=False
    )

    print(f"Training batches: {len(train_gen)}")
    print(f"Validation batches: {len(val_gen)}")

    # Build model with correct input shape
    input_shape = (CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], CONFIG['CHANNELS'])
    print(f"Building model with input shape: {input_shape}")
    model = ModelBuilder.build_improved_model(input_shape)

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=CONFIG['LEARNING_RATE']),
        loss=focal_loss(alpha=0.7, gamma=2.0),
        metrics=['accuracy', precision_m, recall_m, f1_m]
    )

    print("Model compiled successfully")
    model.summary()

    # Callbacks
    callbacks = [
        ModelCheckpoint(
            'best_landslide_model.keras',
            monitor='val_f1_m',
            mode='max',
            save_best_only=True,
            verbose=1
        ),
        EarlyStopping(
            monitor='val_f1_m',
            mode='max',
            patience=CONFIG['PATIENCE'],
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-7,
            verbose=1
        )
    ]

    # Calculate class weights
    y_train = train_df['label'].values[train_idx]
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
    print(f"Class weights: {class_weight_dict}")

    # Train model
    print("Starting training...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=CONFIG['EPOCHS'],
        callbacks=callbacks,
        class_weight=class_weight_dict,
        verbose=1
    )

    # Load best model
    model.load_weights('best_landslide_model.keras')

    # Threshold optimization
    print("\nOptimizing classification threshold...")
    val_predictions = []
    val_labels = []

    for i in range(len(val_gen)):
        batch_x, batch_y = val_gen[i]
        pred_batch = model.predict(batch_x, verbose=0)
        val_predictions.extend(pred_batch.flatten())
        val_labels.extend(batch_y.flatten())

    # Additional debugging for threshold optimization
    print(f"Prediction probability range: {y_probs.min():.4f} to {y_probs.max():.4f}")
    print(f"Prediction probability mean: {y_probs.mean():.4f}")
    print(f"True label distribution: {np.bincount(y_true.astype(int))}")

    # If no good threshold found, use default evaluation
    if not best_metrics:
        print("Warning: No optimal threshold found, using default 0.5")
        best_thresh = 0.5
        y_pred_default = (y_probs > 0.5).astype(int)
        if len(np.unique(y_pred_default)) > 1:
            f1_default = f1_score(y_true, y_pred_default, average='weighted', zero_division=0)
            best_metrics = {
                'threshold': 0.5,
                'weighted_f1': f1_default,
                'class_0_f1': 0.0,
                'class_1_f1': 0.0
            }
        else:
            best_metrics = {
                'threshold': 0.5,
                'weighted_f1': 0.0,
                'class_0_f1': 0.0,
                'class_1_f1': 0.0
            }

    # Find optimal threshold using F1 score
    thresholds = np.arange(0.1, 0.9, 0.01)
    best_f1 = 0
    best_thresh = 0.5
    best_metrics = {}

    for thresh in thresholds:
        y_pred = (y_probs > thresh).astype(int)

        # Skip thresholds that result in all one class
        if len(np.unique(y_pred)) < 2:
            continue

        try:
            f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

                # Calculate per-class F1 scores safely
                report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
                best_metrics = {
                    'threshold': thresh,
                    'weighted_f1': f1,
                    'class_0_f1': report.get('0', {}).get('f1-score', 0.0),
                    'class_1_f1': report.get('1', {}).get('f1-score', 0.0)
                }
        except Exception as e:
            print(f"Error at threshold {thresh}: {e}")
            continue

    print(f"\nBest threshold: {best_thresh:.3f}")
    print(f"Weighted F1: {best_metrics.get('weighted_f1', 0.0):.4f}")
    print(f"Class 0 F1: {best_metrics.get('class_0_f1', 0.0):.4f}")
    print(f"Class 1 F1: {best_metrics.get('class_1_f1', 0.0):.4f}")

    # Final evaluation
    y_pred_final = (y_probs > best_thresh).astype(int)
    print(f"\nFinal Classification Report:")
    print(classification_report(y_true, y_pred_final, target_names=['No Landslide', 'Landslide'], zero_division=0))

    # Plot training history
    plot_training_history(history)

    # Generate test predictions
    generate_test_predictions(model, best_thresh)

    print("Training completed successfully!")
    return model, best_thresh

def plot_training_history(history):
    """Plot training metrics"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    metrics = ['accuracy', 'loss', 'f1_m', 'precision_m']
    titles = ['Accuracy', 'Loss', 'F1 Score', 'Precision']

    for i, (metric, title) in enumerate(zip(metrics, titles)):
        row, col = i // 2, i % 2

        if metric in history.history:
            axes[row, col].plot(history.history[metric], label=f'Training {title}')
            if f'val_{metric}' in history.history:
                axes[row, col].plot(history.history[f'val_{metric}'], label=f'Validation {title}')

            axes[row, col].set_title(title)
            axes[row, col].set_xlabel('Epoch')
            axes[row, col].set_ylabel(title)
            axes[row, col].legend()

    plt.tight_layout()
    plt.show()

def generate_test_predictions(model, threshold):
    """Generate test predictions"""
    test_df = pd.read_csv(CONFIG['TEST_CSV'])
    test_ids = test_df['ID'].values

    predictions = []
    batch_size = CONFIG['BATCH_SIZE']

    print(f"Generating predictions for {len(test_ids)} test images...")

    for i in range(0, len(test_ids), batch_size):
        batch_ids = test_ids[i:i+batch_size]
        batch_imgs = []

        for img_id in batch_ids:
            img = DataProcessor.load_and_normalize_image(img_id, CONFIG['TEST_DATA'])
            if img is not None:
                batch_imgs.append(img)
            else:
                batch_imgs.append(np.zeros((CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'], CONFIG['CHANNELS'])))

        batch_imgs = np.array(batch_imgs)
        probs = model.predict(batch_imgs, verbose=0).flatten()
        preds = (probs > threshold).astype(int)
        predictions.extend(preds)

        if (i // batch_size + 1) % 10 == 0:
            print(f"Processed {min(i+batch_size, len(test_ids))}/{len(test_ids)}")

    # Create submission
    submission_df = pd.DataFrame({
        'ID': test_ids,
        'label': np.array(predictions, dtype=int)
    })

    submission_df.to_csv('optimized_submission.csv', index=False)
    print(f"Submission saved. Prediction distribution: {Counter(predictions)}")

if __name__ == "__main__":
    # Clear memory
    tf.keras.backend.clear_session()
    gc.collect()

    # Run main training
    model, best_threshold = main()

Mounted at /content/drive
Starting optimized landslide detection training...
Loaded 7147 training samples
Class distribution: {0: 5892, 1: 1255}
Checking image sizes...
Image ID_HUD1ST: shape (64, 64, 12)
Image ID_KGE2HY: shape (64, 64, 12)
Image ID_VHV9BL: shape (64, 64, 12)
Image ID_ZT0VEJ: shape (64, 64, 12)
Image ID_5NFXVY: shape (64, 64, 12)
Balanced dataset: 9426 samples
Training batches: 590
Validation batches: 90
Building model with input shape: (64, 64, 12)
Model compiled successfully


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 64, 64,    │          0 │ -                 │
│ (InputLayer)        │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 64, 64,    │      9,632 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64, 64,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 32, 32,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 32, 32,    │      2,400 │ max_pooling2d[0]… │
│ (SeparableConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ separable_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 32, 32,    │      4,736 │ activation_1[0][… │
│ (SeparableConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 32, 32,    │      2,112 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ separable_conv2d… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 32, 32,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 32, 32,    │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 16, 16,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 16, 16,    │          0 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_2  │ (None, 16, 16,    │      8,896 │ dropout[0][0]   

 Total params: 899,329 (3.43 MB)

 Trainable params: 892,737 (3.41 MB)

 Non-trainable params: 6,592 (25.75 KB)

Class weights: {0: np.float64(0.6065138977296839), 1: np.float64(2.8471115537848606)}
Starting training...
Epoch 1/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.5482 - f1_m: 7.9671 - loss: 0.4236 - precision_m: 7.9698 - recall_m: 8.3835
Epoch 1: val_f1_m improved from -inf to 4.26470, saving model to best_landslide_model.keras
590/590 ━━━━━━━━━━━━━━━━━━━━ 225s 339ms/step - accuracy: 0.5483 - f1_m: 7.9676 - loss: 0.4235 - precision_m: 7.9698 - recall_m: 8.3844 - val_accuracy: 0.4629 - val_f1_m: 4.2647 - val_loss: 0.0608 - val_precision_m: 2.7889 - val_recall_m: 10.5889 - learning_rate: 1.0000e-04
Epoch 2/100
590/590 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - accuracy: 0.6382 - f1_m: 8.7426 - loss: 0.2578 - precision_m: 8.0198 - recall_m: 10.0053
Epoch 2: val_f1_m did not improve from 4.26470
590/590 ━━━━━━━━━━━━━━━━━━━━ 183s 310ms/step - accuracy: 0.6382 - f1_m: 8.7427 - loss: 0.2578 - precision_m: 8.0197 - recall_m: 10.0056 - val_accuracy: 0.5727 - val_f1_m: 4.1121 - val_loss:

NameError: name 'y_probs' is not defined